In [ ]:
# Imports and helpers
import requests
import json
import math
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean

QEC_URL = 'http://localhost:5001'  # qec_stub default

def submit_qec_job(measurements, code='repetition'):
    payload = { 'code': code, 'measurements': measurements }
    r = requests.post(QEC_URL + '/api/qec/submit', json=payload)
    r.raise_for_status()
    return r.json().get('job_id')

def check_qec_job(job_id):
    r = requests.get(QEC_URL + f'/api/qec/status/{job_id}')
    r.raise_for_status()
    return r.json()

def get_qec_result(job_id):
    r = requests.get(QEC_URL + f'/api/qec/result/{job_id}')
    if r.status_code == 200:
        return r.json()
    return None

In [ ]:
# Simple measurement + noise models and decoders
def simulate_qubit_syndrome(true_bit, p_flip, rounds=3):
    # returns list of measurements: each is 0/1 possibly flipped by noise probability p_flip
    return [ int(np.random.rand() < (true_bit ^ (np.random.rand() < p_flip))) for _ in range(rounds) ]

def repetition_decode(measurements):
    # majority vote
    return 1 if sum(measurements) > len(measurements)/2 else 0

def compute_error_rate(trials, p_flip, rounds=3):
    errors = 0
    for _ in range(trials):
        true = np.random.randint(0,2)
        meas = simulate_qubit_syndrome(true, p_flip, rounds=rounds)
        dec = repetition_decode(meas)
        if dec != true: errors += 1
    return errors / trials

# Quick sanity check
print('Repetition code error rate example:', compute_error_rate(1000, 0.1, rounds=3))

In [ ]:
# Parameter sweep: how effective is repetition decoding vs raw flip probability
ps = np.linspace(0.01, 0.3, 10)
err_raw = []
err_rep = []
for p in ps:
    # raw single-measure error = p (if we assume a single sample equals true_bit flipped with p)
    err_raw.append(p)
    err_rep.append(compute_error_rate(2000, p, rounds=3))

plt.plot(ps, err_raw, label='raw')
plt.plot(ps, err_rep, label='repetition (3 rounds)')
plt.xlabel('Flip probability p')
plt.ylabel('Logical error rate')
plt.legend()
plt.title('Repetition code vs raw error')
plt.show()

## Using the QEC REST stub for a submission example
The next cell builds a small measurement payload and submits it to `qec_stub.py`. The stub returns a job id and a dummy correction result which this notebook fetches and prints.

In [ ]:
# Build a tiny measurement payload to submit to qec_stub
measurements = [ {'qubit': 0, 'basis': 'Z', 'round': 0, 'value': 1}, {'qubit': 0, 'basis': 'Z', 'round': 1, 'value': 0} ]
try:
    job = submit_qec_job(measurements, code='repetition')
    print('submitted job', job)
    import time
    for _ in range(10):
        st = check_qec_job(job)
        print('status', st)
        if st.get('status') == 'done': break
        time.sleep(0.2)
    res = get_qec_result(job)
    print('result', res)
except Exception as e:
    print('QEC submission failed (is qec_stub running?):', e)